In [140]:
import pandas as pd
import nltk 
from typing import List
import spacy
import json
import ast
import os
import re

In [100]:
nlp = spacy.load("fr_core_news_lg")

In [101]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

True

In [167]:
current_dir = os.path.dirname(os.path.realpath("__file__"))
repo_dir = os.path.dirname(current_dir)
DATA_FILE = os.path.join(repo_dir,"data","recettes.csv")
data = pd.read_csv(DATA_FILE, encoding="utf-8",sep=',')

In [ ]:
data["Etapes"][0]

0        [{'Étape 1': "Enlever l'écorce et les pépins d...
1        [{'Étape 1': 'Pelez et émincez les oignons.'},...
2        [{'Étape 1': 'Couper la chair du potiron en gr...
3        [{'Étape 1': 'Couper le butternut en gros cube...
4        [{'Étape 1': 'Préchauffer le four à 210°C (the...
                               ...                        
47335    [{'Étape 1': "Transvaser le litre d'alcool dan...
47336    [{'Étape 1': 'Dans un shaker, mélanger tous le...
47337    [{'Étape 1': 'Commencer par réaliser un sirop ...
47338    [{'Étape 1': 'Verser les glaçons directement d...
47339    [{'Étape 1': 'Préparez directement dans un gra...
Name: Etapes, Length: 47340, dtype: object

In [124]:
def split_quantity_units(text):
    if re.match(r"^\d.*[a-zA-Z]$",text) is not None:
        my_list = re.split(r'(\d+)', text)
        return " ".join(my_list)
    return text

In [160]:
def rm_duplicated_dgt(text:str):
    if text.isnumeric() and len(text)%2==0 and text[:int(len(text)/2)]==text[int(len(text)/2):]:
        return(text[:int(len(text)/2)])
    else:
        return(text)

# rm_duplicated_dgt(text)

In [168]:
def spacy_prep(texts):
    cleaned_texts = []
    docs = list(nlp.pipe(texts))
    for doc in docs:
        cleaned_texts.append(" ".join([token.lemma_ for token in doc if not token.is_punct
                                                                            # and not tokens.is_digit
                                                                            # and not token.is_oov
                                                                            and not token.is_stop
                                                                            ]))
    return cleaned_texts

In [169]:
def join_qty_ingredients(ingredients:str):
    list_ingredients = ast.literal_eval(ingredients)
    new_ingredients = []
    for i in range(len(list_ingredients)-1):
        if bool(re.search(r'\d', list_ingredients[i])):
            list_ingredients[i] = rm_duplicated_dgt(list_ingredients[i])
            list_ingredients[i] = split_quantity_units(list_ingredients[i])
            if list_ingredients[i+1] != list_ingredients[i].split(" ")[-1] and list_ingredients[i+1] != "":
                new_ingredients.append(" ".join([list_ingredients[i],list_ingredients[i+1]]))
            else:
                if i+2 <= len(list_ingredients):
                    new_ingredients.append(" ".join([list_ingredients[i],list_ingredients[i+2]]))
    return ", ".join(new_ingredients)

In [ ]:
unit_abbreviation = {
    "c.à.s" : "cuillère à soupe",
    "c.à.c" : "cuillère à café",
    "g"     : "gramme",
    "l"     : "litre",
    "cl"    : "centilitre",
    "ml"    : "millilitre",
    "kg"    : "kilogramme",
}

In [162]:
print(join_qty_ingredients(data["Ingredients"][20]))

 1 kg de champignon « trompettes de la mort »,  2 c.à.s d'huile d'olive ,  1 gousse d'ail hachée,  1 ⁄ 2 botte de persil haché


In [ ]:
def unit_full_name():
    

In [170]:
def remove_accents(text:str):
    text = re.sub(r'[àáâãäå]', 'a', text)
    text = re.sub(r'[èéêë]', 'e', text)
    text = re.sub(r'[ìíîï]', 'i', text)
    text = re.sub(r'[òóôõö]', 'o', text)
    text = re.sub(r'[ùúûü]', 'u', text)
    return(text)

In [171]:
def preprocessing_fn(df:pd.DataFrame):
    data = df.copy()
    TEXT_FEATURES = ["Nom","Type","Ingredients","Ustensiles","Etapes"]
    for key in TEXT_FEATURES:
        data[key] = data[key].apply(lambda x:x.lower())
        if key == "Ingredients":
            data[key] = data[key].apply(lambda x:join_qty_ingredients(x))
        data[key] = spacy_prep(data[key].to_numpy())
        data[key] = data[key].apply(lambda x: remove_accents(x))
        # data[key] = data[key].apply(lambda x: rm_duplicated_dgt(x))
    
    return data

In [172]:
clean

,Nom,Duree,Personnes,Type,Ingredients,Ustensiles,Etapes
0,Soupe veloutée de potimarron et pommes de terre,45 min,4,Entrée,"['22', 'pommes de terre ', '22', 'oignons hach...","['louche', 'mixeur plongeant', 'Blender chauff...","[{'Étape 1': ""Enlever l'écorce et les pépins d..."
1,Soupe à l'oignon,45 min,4,Entrée,"['44', 'oignons ', '50g', 'de beurre ', '1c.à....","['louche', 'mixeur plongeant', 'Blender chauff...","[{'Étape 1': 'Pelez et émincez les oignons.'},..."
2,Velouté de potiron,1 h 15 min,4,Entrée,"['11', 'oignon ', '', 'poivre ', '', 'sel ', '...","['louche', 'mixeur plongeant', 'Couteau', 'Coc...",[{'Étape 1': 'Couper la chair du potiron en gr...
3,Velouté de butternut à la cannelle,35 min,4,Entrée,"['11', 'oignon ', '1c.à.s', 'c.à.s', 'de crème...","['louche', 'mixeur plongeant', 'Couteau', 'Cui...",[{'Étape 1': 'Couper le butternut en gros cube...
4,Tarte aux poireaux et aux lardons,50 min,6,Entrée,"['250g', 'de farine ', '140g', 'de beurre ', '...","['Moule à tarte', 'Four', 'couteau', 'pinceau'...",[{'Étape 1': 'Préchauffer le four à 210°C (the...
...,...,...,...,...,...,...,...
47335,Liqueur de Génépi,10 min,1,Boisson,"['40morceaux', 'de sucre ', '40brins', 'de gén...","['Entonnoir', 'balance de cuisine']","[{'Étape 1': ""Transvaser le litre d'alcool dan..."
47336,Caipirinha,3 min,1,Boisson,"['1dose', 'de jus de citron vert frais (15 ml)...","['Cuillère en bois', 'shaker']","[{'Étape 1': 'Dans un shaker, mélanger tous le..."
47337,Kombucha maison au citron et au gingembre,-,1,Boisson,"['1morceau', 'de gingembre râpé', '100g', 'de ...","['casserole', 'balance de cuisine']",[{'Étape 1': 'Commencer par réaliser un sirop ...
47338,Gin tonic,5 min,1,Boisson,"['4cl', 'de gin ', '2rondelles', 'de citron ',...",[],[{'Étape 1': 'Verser les glaçons directement d...


In [173]:
cleaned_data = preprocessing_fn(data)

In [179]:
final_data = cleaned_data.loc[:,["Nom","Ingredients","Ustensiles","Etapes"]]
final_data.to_csv(os.path.join(repo_dir,"data","cleaned_data.csv"),encoding="utf-8",sep=",", index=False)